# Proyecto K-Nearest Neighbours

In [1]:
# Librería para la declaración y uso de Data Frames:
import pandas as pd
# Configuración de pandas para mostrar todas las columnas del DataFrame sin truncarlas al visualizarlo
pd.set_option('display.max_columns', None);

# Librería para poder utilizar la base de datos SQLite:
import sqlite3

# Librería para poder trabajar con datos en formato json:
import json

# Librería para poder trabajar con un vectorizador tipo TfidVectorizer:
from sklearn.feature_extraction.text import TfidfVectorizer

# ALibrería para calcular la distancia de coseno:
from sklearn.metrics.pairwise import cosine_similarity

# Librería para utilizar el modelo KNN:
from sklearn.neighbors import NearestNeighbors

## Paso 1 - Lectura de Datos:

En primer lugar, es necesario **leer y guardar la información** en dos variables para poder empezar a trabajar con ellas.

Para ello, se has guaradado los archivos con todos los datos en la ruta: */workspaces/KNN-clara-ab/data/raw/tmdb_5000_credits.csv* y */workspaces/KNN-clara-ab/data/raw/tmdb_5000_movies.csv* y se han cargado en dos Data Frames:

In [2]:
# Lectura del CSV con los datos, dada la ruta donde se guarda el archivo:
df_movies = pd.read_csv ('/workspaces/KNN-clara-ab/data/raw/tmdb_5000_movies.csv');

# Se muestran las 2 primeras filas del Data Frame
df_movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [3]:
# Lectura del CSV con los datos, dada la ruta donde se guarda el archivo:
df_credits = pd.read_csv ('/workspaces/KNN-clara-ab/data/raw/tmdb_5000_credits.csv');

# Se muestran las 5 primeras filas del Data Frame
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Paso 2 - Creación de una base de datos:

Tal y como se indica en las instrucciones del proyecto, se debe **crear una base de datos** para almacenar los dos DataFrames en tablas disintas. 

Para ello, se va a utilizar **SQLite3** dado que **no se requiere un servidor independiente** y permite estructurar los datos provenientes de los conjuntos de datos, facilitanto su posterior análisis. En este caso, se va a emplear el **método**  `.connect()` de forma que se **establezca la conexión** con una base de datos almancenada en la ruta especificada o **crearla** en caso de que todavía no exista. 

In [4]:
# Se establece la conexión con la base de datos 'movies_db':
conn = sqlite3.connect("/workspaces/KNN-clara-ab/data/raw/movies_db.sqlite");

# Se crea un cursor para ejecutar las consultas:
cursor = conn.cursor();

Una vez está la base de datos creada, simplemente se han de **guardar los DataFrames** como tablas:

In [5]:
# Se guarda el DataFrame de 'movies':
df_movies.to_sql("movies", conn, if_exists="replace", index = False);

# Se guarda el DataFrame de 'credits':
df_credits.to_sql("credits", conn, if_exists="replace", index = False);

# Se comprueba que las tablas se han creado correctamente:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)

,name
0,movies
1,credits


Ahora que ya se tienen los datos guardados como dos tablas separadas, en las instrucciones se indica que se han de **unir ambas dos a partir del título de la película**.

Con este objetivo, se van a **seleccionar todas las columnas** de ambas tablas donde el **valor `title` sea igual en los dos casos**. 

In [ ]:
# Se define una query para unir ambas tablas a partir del título de la película:
query = """
    SELECT *
    FROM movies
    INNER JOIN credits
    ON movies.title = credits.title;
"""

# Se ejecuta la query y se guarda en un DataFrame:
df_final = pd.read_sql(query, conn);

# Se comprueba que se ha realizado correctamente la unión:
df_final.head(2)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [8]:
# Se cierra la conexión: 
conn.close();

A continuación, se indica qué columnas deben quedarse en el DataFrame para trabajar con ellas:  `movie_id`, `title`, `overview`, `genres`, `keywords`, `cast`, `crew`:

In [9]:
# Se seleccioan las columnas necesarias:
df_final = df_final[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]];

# Se comprueba que se ha realizado la acción correctamente:
df_final.head()

,movie_id,title,title,overview,genres,keywords,cast,crew
0,19995,Avatar,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


## Paso 3 - Transformación de los datos:

Gracias a la visualización mostrada, se puede ver que hay algunas **columnas en formato JSON**. Para cada caso, se indica una acción distinta: 

- `genres` y `keywords` : Se selecciona el atributo `name` y se reemplaza el valor

- `cast` : Se seleccionan los tres primeros nombres.

- `crew` : Se transforma para que contenga el nombre del director

- `overview` : Se convierte en una lista

Además, para cada columna, se va a pasar **todo el texto a formato string** para poder trabajar con él de forma segura y en el mismo formato para todos los casos:

In [10]:
# 'genres': Se selecciona el atributo 'name' de cada género:
df_final['genres'] = df_final['genres'].apply(lambda x: [genre['name'] for genre in json.loads(x)]);
df_final["genres"] = df_final["genres"].apply(lambda x: [str(genre) for genre in x]);

 # 'keywords': Se selecciona el atributo 'name' de cada keyword:
df_final['keywords'] = df_final['keywords'].apply(lambda x: [keyword['name'] for keyword in json.loads(x)]);
df_final["keywords"] = df_final["keywords"].apply(lambda x: [str(keyword) for keyword in x]);

# 'cast': Se extraen los tres primeros actores/actrices:
df_final['cast'] = df_final['cast'].apply(lambda x: [actor['name'] for actor in json.loads(x)[:3]]);
df_final["cast"] = df_final["cast"].apply(lambda x: [str(actor) for actor in x]);

# 'crew': Se transforma para que contenga el nombre del director:
df_final['crew'] = df_final['crew'].apply(lambda x: " ".join([crew_member['name'] for crew_member in json.loads(x) if crew_member['job'] == 'Director']));
df_final["crew"] = df_final["crew"].apply(lambda x: [str(crew_member) for crew_member in x]);

# 'overview': Se convierte en una lista:
df_final['overview'] = df_final['overview'].apply(lambda x: [x]);
df_final["overview"] = df_final["overview"].apply(lambda x: [str(x)]);

Por último, se pide reducir el conjunto de datos combinando las columnas en una sola llamada `tags`, donde todos los elementos han de ir **separados por espacios en blanco**:

In [13]:
# Se combinan todas las columnas en una sola: 
df_final["tags"] = df_final["overview"] + df_final["genres"] + df_final["keywords"] + df_final["cast"] + df_final["crew"];

# Se une todo por comas:
df_final['tags'] = df_final['tags'].apply(lambda x: ",".join(x));

# Se reemplazan por espacios en blanco:
df_final['tags'] = df_final['tags'].apply(lambda x: x.replace(",", " "));

# Se realiza una copia del data frame final para poder tener una versión con todas las columnas y otra procesada solo con la columna 'tags':
df_processed = df_final.copy();

# Se eliminan todas las columnas menos 'tags';
df_processed.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True);

# Se comprueba si el resultado es el esperado: 
df_processed.iloc[0].tags


"['In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization.'] Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver J a m e s   C a m e r o n"

## Paso 4 - Vectorización: 

Los algoritmos de *Machine Learning* **no pueden trabajar directamente con datos en formato texto**. Por esta razón, en general, se suele realizar una codificación de variables categóricas a numéricas. Sin embargo, en este caso, la única variable se encuentra en formato texto por lo que no tendría sentido cambiarlo. 

Por este motivo, se va a utilizar una instancia de la clase `TfidVectorizer()`, **transformando así todos los textos en una matriz de valores TF-IDF** (*Term Frequency - Inverse Document Frequenc*y) que permita **evaluar la importacia de cada palabra en un documento en relación con el resto**. De esta forma, se consigue **reducir el peso de las palabras más comunes** y se **destacan las más relevantes** para el análisis. 

In [14]:
# Se crea una instrancia del vectorizador:
vectorizer = TfidfVectorizer();

# Se aplica la vectorización sobre la columna 'tags':
vec_matrix = vectorizer.fit_transform(df_processed["tags"]);

## Paso 4 - Construcción KNN:

Ahora que ya se tiene el conjunto de datos preparado, se puede pasar a emplear el algoritmo de *Machine Learning* **K-Nearest Neighbours**, basado en la idea de que una **observación nueva** se puede clasificar en función de la **similitud con sus *vecinos más cercanos***.

En las instrucciones del proyecto, se indica que la distancia a utilizar es la **distancia coseno**, por lo que, a la hora de generar el modelo se va a indicar que la `metric` es `cosine`:

In [18]:
# Se genera el modelo:
knn_model = NearestNeighbors (n_neighbors = 6, algorithm = "brute", metric = "cosine");

# Se entrena el modelo:
knn_model.fit(vec_matrix);

Finalmente se pide diseñar una función de similitud basada en la distancia coseno definida. El **objetivo final** de esta función es **recomendar películas similares a una película que el usaurio indique** como input de la función. 

In [21]:
# # # Función de Recomendación de Películas # # #
def recommend_movies(movie):
   
    # Se busca el índice de la película dada como input: 
    movie_index = df_processed[df_processed["title"] == movie].index[0];
    
    # Se aplica la vectorización:
    movie_vector = vec_matrix[movie_index];
    
    # Se obtienen los 5 vecinos más cercanos:
    distances, indices = knn_model.kneighbors(movie_vector);
    
    # Se crea una la lista con los títulos recomendados, sin incluir la misma (se empieza el bucle en 1):
    recommended_titles = [];
    for i in range(1, len(indices[0])):
        recommended_titles.append(df_processed.iloc[indices[0][i]]['title']);
    
    return recommended_titles

In [22]:
# Se elimina cualquier duplicado para trabajar sin problemas: 
df_processed = df_processed.loc[:, ~df_processed.columns.duplicated()];

# Se invoca a la función y se guarda el output en una variable:
recommended_movies = recommend_movies("How to Train Your Dragon");

# Se muestran las películas recomendadas:
for movie in recommended_movies: print(movie);

How to Train Your Dragon 2
Dragon Nest: Warriors' Dawn
Pete's Dragon
George and the Dragon
Eragon
